In [ ]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [1]:
# Install necessary libraries
!pip install transformers[torch]
!pip install accelerate -U
!pip install scikit-learn pandas numpy

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_

In [2]:
!pip install --upgrade transformers

In [3]:
# General purpose libraries
import numpy as np
import pandas as pd
import re
import random
import torch

In [4]:
# Scikit-learn for classical ML
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import f1_score


In [13]:
# Transformers for deep learning
import transformers
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

In [14]:
# Set a seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [26]:
# Define the pre-trained model we'll use
MODEL_NAME = 'roberta-base'

In [15]:
# Load the datasets
train_df = pd.read_csv("/kaggle/input/mc-datathon-2025-sentiment-analysis/train.csv")
test_df = pd.read_csv("/kaggle/input/mc-datathon-2025-sentiment-analysis/test.csv")


In [16]:
# Display basic information
print("Train Data Shape:", train_df.shape)
print("Test Data Shape:", test_df.shape)
print("\nTrain Data Head:")
print(train_df.head())

Train Data Shape: (7633, 3)
Test Data Shape: (5205, 2)

Train Data Head:
   id                                               text sentiment
0   0  Extremely easy user interface. Helpsnin using ...  positive
1   1   Hey Luis, thanks!  I have Flash and my prof`s...  positive
2   2   For sure you should continue to Tweet WHILE o...  negative
3   3                    on the phone w. Chantellie ! <3   neutral
4   4  Clunky and very slow to load. Not fun at all -...  negative


In [17]:
# Check sentiment distribution
print("\nSentiment Distribution:")
print(train_df['sentiment'].value_counts())


Sentiment Distribution:
sentiment
neutral     2896
positive    2513
negative    2223
Name: count, dtype: int64


In [18]:
# Check for missing values
print("\nMissing values in train:", train_df.isnull().sum().sum())
print("Missing values in test:", test_df.isnull().sum().sum())


Missing values in train: 1
Missing values in test: 0


In [19]:
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove user @ mentions and '#' from tweet
    text = re.sub(r'\@\w+|\#','', text)
    # Remove special characters and numbers, keeping only letters and spaces
    text = re.sub(r'[^A-Za-z\s]+', '', text)
    # Convert to lowercase
    text = text.lower()
    return text.strip()

# Apply preprocessing
train_df['clean_text'] = train_df['text'].apply(preprocess_text)
test_df['clean_text'] = test_df['text'].apply(preprocess_text)

# Map sentiment labels to integers
sentiment_map = {'positive': 0, 'neutral': 1, 'negative': 2}
train_df['label'] = train_df['sentiment'].map(sentiment_map)

print("\nPreprocessed Data Head:")
print(train_df[['text', 'clean_text', 'label']].head())


Preprocessed Data Head:
                                                text  \
0  Extremely easy user interface. Helpsnin using ...   
1   Hey Luis, thanks!  I have Flash and my prof`s...   
2   For sure you should continue to Tweet WHILE o...   
3                    on the phone w. Chantellie ! <3   
4  Clunky and very slow to load. Not fun at all -...   

                                          clean_text  label  
0  extremely easy user interface helpsnin using t...    0.0  
1  hey luis thanks  i have flash and my profs cod...    0.0  
2  for sure you should continue to tweet while on...    2.0  
3                          on the phone w chantellie    1.0  
4  clunky and very slow to load not fun at all  j...    2.0  


In [29]:
# ==============================================================================
# STEP 4: CREATE PYTORCH DATASET
# ==============================================================================
class SentimentDataset(Dataset):
    """Custom Dataset class for the Trainer."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Create a dictionary for the item at the given index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)



# Re-create your datasets with the updated class
print("Tokenizing data for RoBERTa...")
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
train_encodings = tokenizer(train_df['clean_text'].tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_df['clean_text'].tolist(), truncation=True, padding=True, max_length=128)

train_dataset = SentimentDataset(train_encodings, train_df['label'].tolist())
test_dataset_for_prediction = SentimentDataset(test_encodings, [0] * len(test_df))
print("Tokenization complete.")

Tokenizing data for RoBERTa...
Tokenization complete.


In [20]:
# STEP 3: Print the version to confirm it's a new one (should be 4.x.x)
print("---")
print("Transformers version:", transformers.__version__)
print("---")

---
Transformers version: 4.53.3
---


In [ ]:


# Load Model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

# Define F1 score metric for evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    return {'f1': f1}
# Define Training Arguments (for modern transformers versions)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    
    # --- Modern arguments below ---
    eval_strategy="epoch",    # Use this instead of evaluate_during_training
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    
    report_to="none"
)

 # STEP 5: Confirmation message
print("TrainingArguments created successfully!")
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset, # We evaluate on the training set for this example
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Get predictions for the test set
test_predictions = trainer.predict(SentimentDataset(test_encodings, [0]*len(test_df)))
roberta_preds = np.argmax(test_predictions.predictions, axis=1)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TrainingArguments created successfully!


Epoch,Training Loss,Validation Loss


In [ ]:
print("Generating predictions on the test set...")
# Get predictions from the trained model
test_predictions = trainer.predict(test_dataset_for_prediction)
# The actual predictions are the index of the highest probability
roberta_preds = np.argmax(test_predictions.predictions, axis=1)

In [ ]:
# Map the integer predictions back to string labels
reverse_sentiment_map = {0: 'positive', 1: 'neutral', 2: 'negative'}
submission_sentiments = [reverse_sentiment_map[pred] for pred in roberta_preds]

In [ ]:
# Create the submission DataFrame
submission_df = pd.DataFrame({'id': test_df['id'], 'sentiment': submission_sentiments})

In [ ]:
# Save to a .csv file
submission_df.to_csv('submission.csv', index=False)
print("\nSubmission file 'submission.csv' created successfully!")
print("Submission file head:")
print(submission_df.head())